In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/databbva2023/digital.csv
/kaggle/input/databbva2023/Diccionario de datos.xlsx
/kaggle/input/databbva2023/archive/sample_submission.csv
/kaggle/input/databbva2023/archive/balances.csv
/kaggle/input/databbva2023/archive/universe_train.csv
/kaggle/input/databbva2023/archive/movements.csv
/kaggle/input/databbva2023/archive/liabilities.csv
/kaggle/input/databbva2023/archive/universe_test.csv
/kaggle/input/databbva2023/archive/customers.csv


In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
import gc

In [3]:
%%time
path0 = '/kaggle/input/databbva2023/archive/'
train = pd.read_csv(f'{path0}universe_train.csv')
test = pd.read_csv(f'{path0}universe_test.csv')
test['attrition'] = -1
universe = pd.concat([train , test[train.columns]])
print(universe.period.value_counts(1).sort_index())
print(train.shape, test.shape, universe.shape)

universe = universe.sample(frac = .2, random_state = 123)

print(train.shape, test.shape, universe.shape)

universe.head()

period
period_1    0.106263
period_2    0.107940
period_3    0.109931
period_4    0.112305
period_5    0.113070
period_6    0.112493
period_7    0.111557
period_8    0.111922
period_9    0.114519
Name: proportion, dtype: float64
(535943, 3) (273636, 3) (809579, 3)
(535943, 3) (273636, 3) (161916, 3)
CPU times: user 1.25 s, sys: 104 ms, total: 1.35 s
Wall time: 1.35 s


,ID,attrition,period
438192,b55e869dd074b9f0928e29af72c6c7066ef71c54efaf76...,0,period_2
21441,5331b971082d83f2dc2e508553449665495cc4ce70368f...,-1,period_9
92583,f8fa6ad731c4e5f9eb8e3e0a1c1364cbf1b0be299ba85f...,0,period_6
75505,712ce7c7823561f8334134f98a584f7226b3199d40fa29...,-1,period_9
208516,10cfb7cda4f6d77d491c069224011c3e87e7bdbd80a388...,0,period_4


In [4]:
%%time


path = f'{path0}movements.csv'


!free -h
df = pd.read_csv(path, sep = ',')
!free -h
print(df.shape)
df = df.loc[ df.ID.isin(universe.ID) ]
print(df.shape)
df.head()

              total        used        free      shared  buff/cache   available
Mem:           31Gi       992Mi        24Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B          0B          0B
              total        used        free      shared  buff/cache   available
Mem:           31Gi       1.5Gi        23Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B          0B          0B
(3697738, 7)
(738560, 7)
CPU times: user 7.8 s, sys: 480 ms, total: 8.28 s
Wall time: 8.86 s


,period,ID,month,type_1,type_2,type_3,type_4
2,period_9,a5b9c4d7ae3dfe5cc1c0ca073b0cb0369e1389bfec5c56...,4,3.6273,3.9726,3.6655,4.4131
9,period_9,c3b3ba594a1d499c3e1fb6bc035c1cef91ff5b2d5a07fd...,11,0.0000,4.7102,3.1109,0.0000
10,period_9,1a410bebc15530d9fd98cf61f399eeda1b617fc6854fd9...,12,4.7829,4.7190,2.9181,2.8271
20,period_9,f7f76c311fbea4809ce68791b6805859c796db0caf0343...,1,4.8076,4.6826,3.1995,3.8209
22,period_9,3db4648b0b31e76cbe2dfd04ae1fa8f28b6a557da154c6...,11,2.8287,0.0000,0.0000,4.4407


In [5]:
%%time

dic_movements0 = {
                    'type_1': ['min', 'mean', 'max', 'sum', 'std'],
                    'type_2': ['min', 'mean', 'max', 'sum', 'std'],
                    'type_3': ['min', 'mean', 'max', 'sum', 'std'],
                    'type_4': ['min', 'mean', 'max', 'sum', 'std'],
                    'month': ['count']
                }

def varMovements(df_movements, historia =  12 , dict_cal = dic_movements0):
    """
    df_movements: base de compras
    historia: cantidad de meses para la historia (máximo 12)
    dict_cal: diccionario con los cálculos a realizar a las variables numéricas
    """
    !free -h
    dfVarMovements = df_movements[df_movements['month'] >= 12 - historia + 1].groupby(['ID', 'period'])\
                        .agg(
                            dict_cal
                        )

    dfVarMovements.columns = [ 'movements_'+ x[1]+'_' + x[0] + '_' + str(historia) for x in dfVarMovements.columns]

    dfVarMovements.reset_index(inplace = True)
    
    return dfVarMovements 


df = varMovements(df, historia = 12 ,  dict_cal  = dic_movements0 )
df.head()

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.7 µs


In [6]:
%%time
dic_movements = {
                    'type_1': ['min', 'mean', 'max', 'sum', 'std'],
                    'type_2': ['min', 'mean', 'max', 'sum', 'std'],
                    'type_3': ['min', 'mean', 'max', 'sum', 'std'],
                    'type_4': ['min', 'mean', 'max', 'sum', 'std'],
                    'month': ['count']
                }

dic_movements6 = {
                    'type_1': ['min', 'mean', 'max', 'sum' ],
                    'type_2': ['min', 'mean', 'max', 'sum' ],
                    'type_3': ['min', 'mean', 'max', 'sum' ],
                    'type_4': ['min', 'mean', 'max', 'sum' ],
                    'month': ['count']
                }

dic_movements3 = {
                    'type_1': [ 'mean',  'sum' ],
                    'type_2': [ 'mean',  'sum' ],
                    'type_3': [ 'mean',  'sum' ],
                    'type_4': [ 'mean',  'sum' ],
                    'month': ['count']
                }

dic_movements1 = {
                    'type_1': [ 'mean'],
                    'type_2': [ 'mean'],
                    'type_3': [ 'mean'],
                    'type_4': [ 'mean'],
                }


universe = universe.merge(varMovements(df, historia = 12 ,  dict_cal  = dic_movements ), 
              how='left', on = ['ID', 'period'] )
universe = universe.merge(varMovements(df, historia = 9 , dict_cal  = dic_movements ), 
              how='left', on = ['ID', 'period'] )
universe = universe.merge(varMovements(df, historia = 6 ,  dict_cal  = dic_movements6 ), 
              how='left', on = ['ID', 'period'] )
universe = universe.merge(varMovements(df, historia = 3 , dict_cal  = dic_movements3 ), 
              how='left', on = ['ID', 'period'] )

universe = universe.merge(varMovements(df, historia = 1 , dict_cal  = dic_movements1 ), 
              how='left', on = ['ID', 'period'] )


universe.head()

              total        used        free      shared  buff/cache   available
Mem:           31Gi       1.4Gi        23Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B          0B          0B
              total        used        free      shared  buff/cache   available
Mem:           31Gi       1.5Gi        23Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B          0B          0B
              total        used        free      shared  buff/cache   available
Mem:           31Gi       1.5Gi        23Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B          0B          0B
              total        used        free      shared  buff/cache   available
Mem:           31Gi       1.5Gi        23Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B          0B          0B
              total        used        free      shared  buff/cache   available
Mem:           31Gi       1.5Gi        23Gi       0.0Ki       6.3Gi        29Gi
Swap:            0B     

,ID,attrition,period,movements_min_type_1_12,movements_mean_type_1_12,movements_max_type_1_12,movements_sum_type_1_12,movements_std_type_1_12,movements_min_type_2_12,movements_mean_type_2_12,...,movements_sum_type_2_3,movements_mean_type_3_3,movements_sum_type_3_3,movements_mean_type_4_3,movements_sum_type_4_3,movements_count_month_3,movements_mean_type_1_1,movements_mean_type_2_1,movements_mean_type_3_1,movements_mean_type_4_1
0,b55e869dd074b9f0928e29af72c6c7066ef71c54efaf76...,0,period_2,0.0,1.7285,3.2596,20.742,1.36356,0.0,3.022542,...,12.0357,0.880833,2.6425,0.577,1.731,3.0,2.9095,3.7968,0.0,0.0
1,5331b971082d83f2dc2e508553449665495cc4ce70368f...,-1,period_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f8fa6ad731c4e5f9eb8e3e0a1c1364cbf1b0be299ba85f...,0,period_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,712ce7c7823561f8334134f98a584f7226b3199d40fa29...,-1,period_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10cfb7cda4f6d77d491c069224011c3e87e7bdbd80a388...,0,period_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
%%time
ii = universe.attrition == -1

columnas_eliminar = ['ID', 'attrition', 'period']
colsfeat = list(universe.columns[~universe.columns.isin(columnas_eliminar)])

X = universe.loc[ ~ii, ['period'] + colsfeat].copy()
y = universe.loc[ ~ii, 'attrition'].copy()
yb = y.copy()
ii = yb == 0
yb.loc[~ii ] = 1


ii = X[colsfeat].isna().sum(axis = 1) == len(colsfeat)
X = X.loc[~ii]
yb = yb.loc[~ii]
y = y.loc[~ii]

print((X.isna().sum()/X.shape[0]).sort_values())



ii = X.period.isin(['period_1', 'period_2', 'period_3', 'period_4'])
X_train, y_train, y_trainM = X.loc[ii, colsfeat].copy(), yb.loc[ii].copy(), y.loc[ii].copy()

ii = X.period.isin(['period_5'])
X_test, y_test, y_testM = X.loc[ii, colsfeat].copy(), yb.loc[ii].copy(), y.loc[ii].copy()

ii = X.period.isin(['period_6'])
X_oot, y_oot, y_ootM = X.loc[ii, colsfeat].copy(), yb.loc[ii].copy(), y.loc[ii].copy()

#print(X_train.head())

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

for c in X_train.columns:
    X_train[c].replace([-np.inf, np.inf], np.nan, inplace=True)
    X_train[c] = pd.to_numeric(X_train[c], errors = 'coerce').fillna(0)
    

scaler = StandardScaler()
scaler.fit(X_train )
dft = scaler.transform(X_train )


period                      0.000000
movements_count_month_12    0.000000
movements_sum_type_4_12     0.000000
movements_max_type_4_12     0.000000
movements_mean_type_4_12    0.000000
                              ...   
movements_count_month_3     0.177593
movements_mean_type_3_1     0.360821
movements_mean_type_1_1     0.360821
movements_mean_type_2_1     0.360821
movements_mean_type_4_1     0.360821
Length: 73, dtype: float64
CPU times: user 335 ms, sys: 49.9 ms, total: 385 ms
Wall time: 385 ms


In [8]:
%%time

def selectec_feature(dft1, y_train1, C = .005, nf = 10, cumsumx = .8, corrth = 0.6):
    clf = LogisticRegression(random_state=0, solver = 'saga', penalty = 'l1', C = C).fit(dft1, y_train1)
    coef = pd.DataFrame(clf.coef_[0], index = colsfeat, columns = ['value'])
    coef = coef.abs().sort_values('value', ascending =  False)

    coef['cumsum'] = coef['value'].cumsum()/coef['value'].sum()

    #print(coef[coef['cumsum']<.8])
    #print(coef[coef['cumsum']<.8].shape)

    coef2 = coef[coef['cumsum']< cumsumx]


    correlation_threshold = corrth 


    features_cumsum = list(coef2.index )
    print(len(features_cumsum))

    feat_correlation  = X_train[features_cumsum].corr() #dcorr


    to_drop = set()

    for i, feature1 in enumerate(feat_correlation.columns):
        for j, feature2 in enumerate(feat_correlation.columns):
            if i < j and abs(feat_correlation.iloc[i, j]) > correlation_threshold:
                # Drop the less important of the two correlated features
                if coef.loc[coef.index == feature1]['value'].values < coef.loc[coef.index == feature2]['value'].values :
                    to_drop.add(feature1)
                else:
                    to_drop.add(feature2)

    # Combine features to drop
    features_to_drop = list(to_drop) #.union(set(low_gain_features)))
    features_to_drop = [c for c in features_cumsum if c in features_to_drop ]
    selected_features = [c for c in features_cumsum if c not in features_to_drop ]
    #print("Features to drop:", features_to_drop)
    print(len(selected_features) ,  len(features_to_drop), len(features_cumsum))
    return selected_features[0:nf]



def selectec_feature_0(dft1, y_train1, C = .005, cumsumx = .8):
    clf = LogisticRegression(random_state=0, solver = 'saga', penalty = 'l1', C = C).fit(dft1, y_train1)
    coef = pd.DataFrame(clf.coef_[0], index = colsfeat, columns = ['value'])
    coef = coef.abs().sort_values('value', ascending =  False)

    coef['cumsum'] = coef['value'].cumsum()/coef['value'].sum()

    #print(coef[coef['cumsum']<.8])
    #print(coef[coef['cumsum']<.8].shape)

    coef2 = coef[coef['cumsum']< cumsumx]

    features_cumsum = list(coef2.index )
    
    
    print(len(features_cumsum))
    return features_cumsum

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.87 µs


In [9]:
%%time
selected_features_liab = selectec_feature_0(dft, y_train, C = .1, cumsumx = .8,)
print(selected_features_liab)


cols = ['ID', 'attrition', 'period'] + selected_features_liab


universe[cols].head()


22
['movements_sum_type_3_12', 'movements_mean_type_2_12', 'movements_sum_type_4_9', 'movements_sum_type_1_12', 'movements_mean_type_1_12', 'movements_sum_type_1_6', 'movements_mean_type_3_12', 'movements_count_month_3', 'movements_sum_type_2_12', 'movements_sum_type_2_9', 'movements_min_type_2_12', 'movements_std_type_1_12', 'movements_min_type_1_12', 'movements_max_type_3_12', 'movements_count_month_6', 'movements_mean_type_1_1', 'movements_mean_type_1_3', 'movements_mean_type_4_1', 'movements_mean_type_2_1', 'movements_min_type_3_12', 'movements_mean_type_3_9', 'movements_count_month_9']
CPU times: user 5.55 s, sys: 2.58 ms, total: 5.55 s
Wall time: 5.55 s


,ID,attrition,period,movements_sum_type_3_12,movements_mean_type_2_12,movements_sum_type_4_9,movements_sum_type_1_12,movements_mean_type_1_12,movements_sum_type_1_6,movements_mean_type_3_12,...,movements_min_type_1_12,movements_max_type_3_12,movements_count_month_6,movements_mean_type_1_1,movements_mean_type_1_3,movements_mean_type_4_1,movements_mean_type_2_1,movements_min_type_3_12,movements_mean_type_3_9,movements_count_month_9
0,b55e869dd074b9f0928e29af72c6c7066ef71c54efaf76...,0,period_2,9.7943,3.022542,9.841,20.742,1.7285,16.641,0.816192,...,0.0,3.6149,6.0,2.9095,2.925633,0.0,3.7968,0.0,0.6866,9.0
1,5331b971082d83f2dc2e508553449665495cc4ce70368f...,-1,period_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f8fa6ad731c4e5f9eb8e3e0a1c1364cbf1b0be299ba85f...,0,period_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,712ce7c7823561f8334134f98a584f7226b3199d40fa29...,-1,period_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10cfb7cda4f6d77d491c069224011c3e87e7bdbd80a388...,0,period_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
! pip install duckdb=='0.9.2'
import duckdb

In [11]:
digital = '/kaggle/input/databbva2023/digital.csv'
sample_submission = '/kaggle/input/databbva2023/archive/sample_submission.csv'
balances='/kaggle/input/databbva2023/archive/balances.csv'
universe_train = '/kaggle/input/databbva2023/archive/universe_train.csv'
movements = '/kaggle/input/databbva2023/archive/movements.csv'
liabilities = '/kaggle/input/databbva2023/archive/liabilities.csv'
universe_test = '/kaggle/input/databbva2023/archive/universe_test.csv'
customers ='/kaggle/input/databbva2023/archive/customers.csv'

In [12]:
%%time
# 
# 'movements_sum_type_4_9'
#movements_std_type_1_12
tabla = movements
historia = 9
var1 = 'type_4'
query2 = f""" with tabla_h1 as ( 
select ID, month, period, {var1} 
from '{tabla}' 
where month>=12-{historia}+1 ),

 tabla_c1 as (
select ID, period, SUM({var1}) as {var1}_{historia}m
from tabla_h1
group by 1, 2 )


select * 
from tabla_c1 


"""
df = duckdb.query(query2).to_df()
print(df.head())
df.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID    period  type_4_9m
0  42bb006636c42857e685d8f2f12931476e6cf4f1a63bde...  period_9     0.0000
1  ad63bc215637a841e4c8a36237ee5092e7c34b5faeaec0...  period_9     0.0000
2  2925c8cb2722ae7bbf4ec0ec57ce278b134e7920c2020d...  period_9    11.7522
3  06dc894987c76123eb639ce26ffc05d29bceb7b897a836...  period_9    31.2719
4  0503d4e31445e638ae852751ae3b952f7a9eef2d3d2ff0...  period_9     5.7476
CPU times: user 8.13 s, sys: 401 ms, total: 8.53 s
Wall time: 2.58 s


,type_4_9m
count,511590.000000
mean,8.622060
std,11.949472
min,0.000000
25%,0.000000
50%,3.007750
75%,13.359150
max,45.000000


In [13]:
%%time
# 
# 'movements_sum_type_4_9'
#movements_std_type_1_12


tabla = movements
historia = 9
var1 = 'type_4'
alias = 'movements'
oper =  'sum' #'stddev_pop'
query2 = f""" with tabla_h1 as ( 
select ID, period, 
{oper}( case when month>=12-{historia} + 1 then {var1} end ) as {alias}_{oper}_{var1}_{historia}m 
from '{tabla}'  
group by 1, 2 )

select * 
from tabla_h1 


"""
df1 = duckdb.query(query2).to_df()
print(df1.head())
df1.describe()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                                                  ID    period  \
0  8ffafd15dcd6b2adda3f852becc5fb12adfc4c23f8774b...  period_1   
1  aec742482e1cf1fed3b67f03c90c36d54d534213af1fac...  period_1   
2  187e5ccd228256788f4abddaeb1a332cac56b8771d25c8...  period_1   
3  69bfbe6420dba38d7de8ef733f262fc703fe84c59fd495...  period_1   
4  84092af1e33501fb21c0fc092133055fbd676294775690...  period_1   

   movements_sum_type_4_9m  
0                   1.9052  
1                   9.5831  
2                   0.0000  
3                   0.0000  
4                   0.0000  
CPU times: user 8.37 s, sys: 364 ms, total: 8.73 s
Wall time: 2.62 s


,movements_sum_type_4_9m
count,511590.000000
mean,8.622060
std,11.949472
min,0.000000
25%,0.000000
50%,3.007750
75%,13.359150
max,45.000000


In [14]:
x = df.merge(df1, on = ['ID', 'period'], how = 'inner')
x

,ID,period,type_4_9m,movements_sum_type_4_9m
0,42bb006636c42857e685d8f2f12931476e6cf4f1a63bde...,period_9,0.0000,0.0000
1,ad63bc215637a841e4c8a36237ee5092e7c34b5faeaec0...,period_9,0.0000,0.0000
2,2925c8cb2722ae7bbf4ec0ec57ce278b134e7920c2020d...,period_9,11.7522,11.7522
3,06dc894987c76123eb639ce26ffc05d29bceb7b897a836...,period_9,31.2719,31.2719
4,0503d4e31445e638ae852751ae3b952f7a9eef2d3d2ff0...,period_9,5.7476,5.7476
...,...,...,...,...
511585,8e153d097168db04d62375020c333ebaa43dc5c41df5ed...,period_7,2.5069,2.5069
511586,66a1b0cfeb7068b9b287fd1f1e345dd3c08bcff436fe7a...,period_7,0.0000,0.0000
511587,07ed07890af69a9489072cd2fa4d31d681a46d5cb62c36...,period_7,0.0000,0.0000
511588,b5fa76b92b6ec7164375e81c2710d25326d5d17b52e4ab...,period_7,3.3211,3.3211


In [ ]:
movements